<a href="https://colab.research.google.com/github/7201krap/PYTORCH_introduction_to_pytorch/blob/main/8_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import torch.nn as nn 
import numpy as np 
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 

In [10]:
# 0. prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target 

n_samples, n_features = X.shape 
print(n_samples, n_features)

X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# scale 
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test  = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

print(y_train.shape)

y_train = y_train.view(y_train.shape[0], 1)
y_test  = y_test.view(y_test.shape[0], 1)


# 1. model 
# f = wx + b, sigmoid at the end 
class LogReg(nn.Module):
    def __init__(self, n_input_features):
        super(LogReg, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted 

model = LogReg(n_features)

# 2. loss and optimizer 
learning_rate = 0.01 
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3. training loop 
num_epochs = 100 
for epoch in range(num_epochs):
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)

    loss.backward()

    optimizer.step()

    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(epoch+1, loss.item())

569 30
torch.Size([455])
10 0.5225288271903992
20 0.45114585757255554
30 0.4014436602592468
40 0.36451447010040283
50 0.3357771635055542
60 0.3126342296600342
70 0.29349958896636963
80 0.2773476541042328
90 0.26348403096199036
100 0.25142088532447815


In [8]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(acc)

tensor(0.8772)
